Соберите случайный ансамбль из нескольких методов выделения факторов - корреляции, взаимной информации, важности признаков, главных компонент, независимых компонент и др. Получите не менее 3 наборов из 5 наиболее важных признаков.

Соберите для каждого набора ансамбль стекинга для задачи, используя (но не ограничиваясь) решающими деревьями, CatBoost, линейной регрессией - всего не менее 3 ансамблей стекинга, каждый из которых состоит из большого числа разнородных моделей.

Используя эти ансамбли, предскажите продолжительность жизни на 2019 год.

* Дополнительно: найдите значения факторов Росстата за 2019 год и предскажите продолжительность жизни на 2020 год.

Данные:
video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv

Итоговый файл с кодом (.py или .ipynb) выложите в github с портфолио.

In [24]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.feature_selection import mutual_info_regression       # взаимная информация
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.decomposition import PCA      # метод главных компонент 
from sklearn.decomposition import FastICA  # метод независимых компонент
from sklearn.decomposition import TruncatedSVD,NMF
from catboost import CatBoostRegressor


In [2]:
def linear_extrapolation (x):
    X = np.array(x.dropna().index.astype(int)).reshape(-1, 1)
    Y = np.array(x.dropna().values).reshape(-1, 1)
    if X.shape[0] > 0:
        f = LinearRegression().fit(X, Y)
        for i in x.index:
            v = x.loc[i]
            if v != v:
                v = f.predict([[int(i)]])[0][0]
                if v < 0:
                    v = 0
                x.loc[i] = v
    return x

def make_ensemble (x, y):
    return {
        "ridge1": np.argsort(np.abs(Ridge(alpha=0.1).fit(x, y).coef_))[::-1],
        "ridge3": np.argsort(np.abs(Ridge(alpha=0.3).fit(x, y).coef_))[::-1],
        "ridge8": np.argsort(np.abs(Ridge(alpha=0.8).fit(x, y).coef_))[::-1],
        "lasso1": np.argsort(np.abs(Lasso(alpha=0.1).fit(x, y).coef_))[::-1],
        "lasso3": np.argsort(np.abs(Lasso(alpha=0.3).fit(x, y).coef_))[::-1],
        "lasso8": np.argsort(np.abs(Lasso(alpha=0.8).fit(x, y).coef_))[::-1]
    }

Загрузка и очистка данных
Линейно интерполируем пропуски, а начальные и конечные пропущенные данные экстраполируем по линейному закону. Удаляем пустые параметры

In [3]:
data = pd.read_csv("https://video.ittensive.com/machine-learning/sc-tatar2020/rosstat/rosstat.csv",
                   na_values=["-", " - ","...","…"," -"])

In [4]:
features = data["feature"]
data.drop(labels=["feature"], inplace=True, axis=1)
data.interpolate(method="linear", axis=1, inplace=True)

In [5]:
data = data.apply(linear_extrapolation, axis=1, result_type="expand")

In [6]:
data["feature"] = features
data.dropna(inplace=True)
features = np.array(data["feature"])

In [7]:
data = data.T[:len(data.columns)-1].astype("float")
data.drop(labels=["2019"], inplace=True)

In [8]:
scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data))
data.columns = features
data.head(10)

,ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.1. ЧИСЛО ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ (на конец года),ОБЩАЯ ХАРАКТЕРИСТИКА ПРЕДПРИЯТИЙ И ОРГАНИЗАЦИЙ 12.2. ОБОРОТ ОРГАНИЗАЦИЙ (миллиардов рублей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.1. Число малых предприятий,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.2. Число малых предприятий на 10 000 человек населения,12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.3. Среднесписочная численность работников (без внешних совместителей),12.3. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.3.4. Оборот малых предприятий,"12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.1. Число малых предприятий в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.2. Число малых предприятий на 10 000 человек населения в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.3. Среднесписочная численность работников (без внешних совместителей) в 2010, 2015 гг.","12.4. ИТОГИ СПЛОШНЫХ НАБЛЮДЕНИЙ 12.4.4. Выручка от реализации товаров (работ, услуг) малых предприятий в 2010, 2015 гг.",...,4.22. ЧИСЛО ФИЛИАЛОВ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года),"4.23. ЧИСЛЕННОСТЬ ПРОФЕССОРСКО-ПРЕПОДАВАТЕЛЬСКОГО ПЕРСОНАЛА , ОСУЩЕСТВЛЯЮЩЕГО ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ПРОГРАММАМ ВЫСШЕГО ОБРАЗОВАНИЯ (на начало учебного года; человек)","4.24. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (на начало учебного года; тысяч человек)","4.25. ЧИСЛЕННОСТЬ СТУДЕНТОВ, ОБУЧАЮЩИХСЯ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ на 10 000 человек населения (на начало учебного года; человек)","4.26. ПРИЕМ НА ОБУЧЕНИЕ ПО ПРОГРАММАМ БАКАЛАВРИАТА, СПЕЦИАЛИТЕТА, МАГИСТРАТУРЫ (тысяч человек)","4.27. ВЫПУСК БАКАЛАВРОВ, СПЕЦИАЛИСТОВ, МАГИСТРОВ (тысяч человек)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.1. Число организаций (на конец года)","4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.2. Численность аспирантов (на конец года; человек)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.1. Число организаций (на конец года)","4.29. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ ДОКТОРАНТОВ 4.29.2. Численность докторантов (на конец года; человек)"
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.5,0.453901
1,0.085716,0.000000,0.008395,0.043234,0.052010,0.000000,0.058462,0.058462,0.058462,0.000000,...,0.950834,0.945835,0.228175,0.225926,0.50625,0.093248,0.166667,0.135710,0.6,0.453901
2,0.160766,0.000000,0.065825,0.098528,0.104020,0.000000,0.116923,0.116923,0.116923,0.000000,...,0.901667,0.891671,0.432540,0.429630,0.50000,0.176849,0.583333,0.269543,0.7,0.489362
3,0.263245,0.023087,0.123254,0.153821,0.156030,0.000000,0.175385,0.175385,0.175385,0.000000,...,0.852501,0.837506,0.638889,0.637037,0.77500,0.292605,0.666667,0.402126,0.6,0.574468
4,0.342423,0.083656,0.180684,0.209115,0.208040,0.000000,0.233846,0.233846,0.233846,0.070488,...,0.850417,0.783342,0.792659,0.792593,0.85000,0.440514,0.833333,0.411507,0.6,0.531915
5,0.450100,0.171797,0.238114,0.264409,0.260049,0.000000,0.292308,0.292308,0.292308,0.141415,...,0.583619,0.729177,0.941468,0.940741,0.95625,0.540193,0.833333,0.376485,0.5,0.496454
6,0.449605,0.208053,0.295543,0.319702,0.312059,0.006489,0.350769,0.350769,0.350769,0.212342,...,0.566945,0.675013,1.000000,1.000000,1.00000,0.710611,1.000000,0.435897,0.5,0.560284
7,0.497923,0.277258,0.352973,0.374996,0.364069,0.082067,0.409231,0.409231,0.409231,0.283268,...,0.583619,0.620848,0.998016,0.996296,0.91875,0.836013,1.000000,0.524078,0.5,0.602837
8,0.596958,0.341322,0.410403,0.430290,0.416079,0.157646,0.467692,0.467692,0.467692,0.354195,...,0.566945,0.566683,0.957341,0.955556,0.63125,0.826367,1.000000,0.668543,0.6,0.609929
9,0.630685,0.342645,0.467832,0.485583,0.468089,0.233224,0.526154,0.526154,0.526154,0.425121,...,0.566945,0.512519,0.922619,0.914815,0.48750,0.900322,0.916667,0.752345,0.7,0.666667


In [9]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

In [10]:
super_features = np.zeros(len(features))
subset_size = 50
subset_power = 100
subset_iter = 50
subset_super = 20
for k in range(subset_super):
    subset_features = np.zeros(len(features))
    for j in range(subset_iter):
        features_ensemble = [0]*len(features)
        for f_random in np.random.randint(len(features), size=[subset_power, subset_size]):
            x_subset = data[features[f_random]].copy()
            if y_column in features[f_random]:
                x_subset = x_subset.drop(labels=[y_column], axis=1)
            for y_column_ in y_columns:
                if y_column_ in features[f_random]:
                    x_subset = x_subset.drop(labels=[y_column_], axis=1)
            ensemble = make_ensemble(x_subset, y)
            for e in ensemble:
                i = 0
                for f in ensemble[e]:
                    features_ensemble[f_random[f]] += subset_size - i
                    i += 1
        subset_features[np.argsort(features_ensemble)[::-1][:10]] += 1
    super_features[np.argsort(subset_features)[::-1][:10]] += 1

In [11]:
print (np.argsort(super_features)[::-1][:5])
print (features[np.argsort(super_features)[::-1][:5]])

[451 162 447 173 452]
['4.8. ВЫПУСК ОБУЧАЮЩИХСЯ ОРГАНИЗАЦИЯМИ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ НАЧАЛЬНОГО, ОСНОВНОГО И СРЕДНЕГО ОБЩЕГО ОБРАЗОВАНИЯ  4.8.2. Выпуск обучающихся с аттестатом о среднем общем образовании  (тысяч человек)'
 'ЧИСЛЕННОСТЬ РАБОТНИКОВ ОРГАНОВ МЕСТНОГО САМОУПРАВЛЕНИЯ 2.17.3. В местных администрациях (исполнительно-распорядительных органах муниципальных образований)'
 '4.5. ЧИСЛЕННОСТЬ УЧИТЕЛЕЙ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ОБРАЗОВАТЕЛЬНУЮ ДЕЯТЕЛЬНОСТЬ ПО ОБРАЗОВАТЕЛЬНЫМ ПРОГРАММАМ НАЧАЛЬНОГО, ОСНОВНОГО И СРЕДНЕГО ОБЩЕГО ОБРАЗОВАНИЯ; 2) (на 20 сентября; тысяч человек)'
 '6.5. ДЕТСКИЙ ОТДЫХ 6.5.2. Численность детей, отдохнувших в них за лето;2) (тысяч человек)'
 '4.9. ЧИСЛО ПРОФЕССИОНАЛЬНЫХ ОБРАЗОВАТЕЛЬНЫХ ОРГАНИЗАЦИЙ, ОСУЩЕСТВЛЯЮЩИХ ПОДГОТОВКУ КВАЛИФИЦИРОВАННЫХ РАБОЧИХ, СЛУЖАЩИХ в 2000-2015 гг. ;2) (на конец года)']


In [12]:
rfr = RandomForestRegressor(n_estimators=250, random_state=17).fit(x, y)

In [13]:
etr = ExtraTreesRegressor(n_estimators=250, random_state=17).fit(x, y)

In [14]:
# Взаимная информация факторов (mutual information) и целевой переменной
mi = mutual_info_regression(x, y)
mi /= np.max(mi)

In [15]:
l1 = []
for column in data.columns:
    if column != y_column and column not in y_columns:
        x = np.array(data[column]).reshape(-1, 1)
        y = data[y_column]
        model = Lasso(alpha=0.01).fit(x, y)
        l1.append(model.score(x, y))
l1 = np.array(l1)

In [16]:
l2 = []
for column in data.columns:
    if column != y_column and column not in y_columns:
        x = np.array(data[column]).reshape(-1, 1)
        y = data[y_column]
        model = Ridge(alpha=0.05).fit(x, y)
        l2.append(model.score(x, y))
l2 = np.array(l2)

In [19]:
ensemble_2 = rfr.feature_importances_ + etr.feature_importances_ + mi + l2 + l1
features[np.argsort(ensemble_2)[::-1][:5]]

array(['18.7. ИСПОЛЬЗОВАНИЕ СЕТИ ИНТЕРНЕТ НАСЕЛЕНИЕМ 18.7.2. Население, использовавшее сеть Интернет каждый день или почти каждый день2) (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общей численности населения соответствующего субъекта Российской Федерации)',
       '13.2. ОБЪЕМ ОТГРУЖЕННЫХ ТОВАРОВ СОБСТВЕННОГО ПРОИЗВОДСТВА, ВЫПОЛНЕННЫХ РАБОТ И УСЛУГ СОБСТВЕННЫМИ СИЛАМИ ПО ВИДАМ ЭКОНОМИЧЕСКОЙ ДЕЯТЕЛЬНОСТИ 13.2.6. «Обеспечение электрической энергией, газом и паром; кондиционирование воздуха» в соответствии с ОКВЭД2 (в фактически действовавших ценах; миллионов рублей)',
       '18.7. ИСПОЛЬЗОВАНИЕ СЕТИ ИНТЕРНЕТ НАСЕЛЕНИЕМ 18.7.1.  Население, использовавшее сеть Интернет2) (по данным выборочного обследования населения по вопросам использования ИКТ; в процентах от общей численности населения соответствующего субъекта Российской Федерации)',
       '12.12. ИТОГИ ВЫБОРОЧНЫХ ОБСЛЕДОВАНИЙ 12.12.3. Объем выручки от продажи товаров, продукции, работ

метод главных компонент (PCA) - 3 модели,
сингулярное разложение (SVD) - 3 модели,
анализ независимых компонент (ICA) - 3 модели,
матричную факторизацию (NMF) - 3 модели.

In [21]:
y_column = "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.1. Все население (число лет)"
y_columns = ["ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.2. Мужчины (число лет)",
            "ОЖИДАЕМАЯ ПРОДОЛЖИТЕЛЬНОСТЬ ЖИЗНИ ПРИ РОЖДЕНИИ 1.16.3. Женщины (число лет)"]
y = data[y_column]
x = data.drop(labels=[y_column], axis=1).drop(labels=y_columns, axis=1)

In [22]:
# метод главных компонент
pca5 = PCA(n_components=5, random_state=42).fit(x)
pca10 = PCA(n_components=10, random_state=42).fit(x)
pca18 = PCA(n_components=18, random_state=42).fit(x)

In [25]:
# сингулярное разложение
svd5 = TruncatedSVD(n_components=5, random_state=42, n_iter=100).fit(x)
svd10 = TruncatedSVD(n_components=10, random_state=42, n_iter=100).fit(x)
svd18 = TruncatedSVD(n_components=18, random_state=42, n_iter=100).fit(x)

In [27]:
# Анализ независимых компонент
ica5 = FastICA(n_components=5, whiten = 'unit-variance', random_state=42).fit(x)
ica10 = FastICA(n_components=10, whiten = 'unit-variance', random_state=42).fit(x)
ica18 = FastICA(n_components=18, whiten = 'unit-variance', random_state=42).fit(x)

In [28]:
# Матричная факторизация
nmf5 = NMF(n_components=5, random_state=42, max_iter=1000).fit(x)
nmf10 = NMF(n_components=10, random_state=42, max_iter=1000).fit(x)
nmf18 = NMF(n_components=18, random_state=42, max_iter=5000).fit(x)

In [29]:
ensemble_3 = np.zeros(len(x.columns))
a = MinMaxScaler().fit_transform(np.array(pca5.components_).T)
print (a.T)

[[0.85785658 0.90516015 0.91595028 ... 0.58885223 0.4328719  0.42671716]
 [0.35707202 0.47843557 0.45876395 ... 0.04614578 0.19769678 0.09131371]
 [0.48682805 0.4758213  0.50294996 ... 0.36127293 0.15709916 0.18011024]
 [0.35840713 0.49004016 0.5079901  ... 0.52040737 0.35170694 0.27873669]
 [0.59229627 0.42792351 0.49410853 ... 0.44426293 0.53137184 0.29081794]]


In [31]:
for i,m in enumerate([pca5, pca10, pca18, svd5, svd10, svd18, ica5, ica10, ica18, nmf5, nmf10, nmf18]):
    a = MinMaxScaler().fit_transform(np.array(m.components_).T)
    for c in a.T:
        if i == 0 or i == 3:
            k = 0.2
        elif i == 1 or i == 4:
            k = 0.4
        elif i == 2 or i == 5:
            k = 0.6
        else:
            k = 1
        ensemble_2 += k*np.abs(c)
features[np.argsort(np.abs(ensemble_3))[::-1][:10]]

array(['4.28. ОРГАНИЗАЦИИ, ВЕДУЩИЕ ПОДГОТОВКУ АСПИРАНТОВ 4.28.1. Число организаций (на конец года)',
       'ЧИСЛЕННОСТЬ РАБОТНИКОВ ОРГАНОВ МЕСТНОГО САМОУПРАВЛЕНИЯ 2.17.3. В местных администрациях (исполнительно-распорядительных органах муниципальных образований)',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.8. ЧИСЛЕННОСТЬ НЕ ЗАНЯТЫХ ТРУДОВОЙ ДЕЯТЕЛЬНОСТЬЮ ГРАЖДАН, ОБРАТИВШИХСЯ ЗА СОДЕЙСТВИЕМ В ПОИСКЕ ПОДХОДЯЩЕЙ РАБОТЫ В ОРГАНЫ СЛУЖБЫ ЗАНЯТОСТИ НАСЕЛЕНИЯ (тысяч человек)',
       'УРОВЕНЬ БЕЗРАБОТИЦЫ 2.9. 1. Уровень безработицы',
       'УРОВЕНЬ БЕЗРАБОТИЦЫ 2.9.2. Уровень  зарегистрированной безработицы',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.10. ПОТРЕБНОСТЬ В РАБОТНИКАХ, ЗАЯВЛЕННАЯ РАБОТОДАТЕЛЯМИ В ОРГАНЫ СЛУЖБЫ ЗАНЯТОСТИ НАСЕЛЕНИЯ (на конец года; человек)',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.11. НАГРУЗКА НЕЗАНЯТОГО НАСЕЛЕНИЯ, СОСТОЯЩЕГО НА РЕГИСТРАЦИОННОМ УЧЕТЕ В ОРГАНАХ СЛУЖБЫ ЗАНЯТОСТИ НАСЕЛЕНИЯ, В РАСЧЕТЕ НА ОДНУ ЗАЯВЛЕННУЮ ВАКАНСИЮ (на конец года; человек)',
       'ЗАНЯТОСТЬ И БЕЗРАБОТИЦА 2.12.

In [32]:
ensemble_2[np.argsort(np.abs(ensemble_2))[::-1][:10]]

array([116.51252838, 112.05921619, 111.99118082, 109.5720808 ,
       109.20568019, 108.58386743, 108.2503292 , 107.34993937,
       107.30514097, 106.7337521 ])

предсказания на 2019 год

In [33]:
x_ = x[:-1]
y_ = np.array(y[1:])

In [34]:
# случайный лес
ensemble_rf = []
for i in range (100):
    ensemble_rf.append(RandomForestRegressor(n_estimators=50, random_state=i).fit(x_, y_))

In [35]:
# сверхслучайные деревья
ensemble_et = []
for i in range (100):
    ensemble_et.append(ExtraTreesRegressor(n_estimators=50, random_state=i).fit(x_, y_))

In [37]:
# CatBoost
ensemble_cb = []
for i in range (100):
    ensemble_cb.append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))

In [38]:
prediction = 0
for i in range (100):
    prediction += ensemble_rf[i].predict(x[-1:])
    prediction += ensemble_et[i].predict(x[-1:])
    prediction += ensemble_cb[i].predict(x[-1:])
prediction /= 300
print (prediction)

[0.90763196]


In [39]:
data_predict = data.copy()
data_predict[y_column] = np.ones(len(data_predict)) * prediction
data_predict = scaler.inverse_transform(data_predict)
answer19 = data_predict[0][np.where(features==y_column)][0]
print('Ожидаемая продолжительность жизни:', round(answer19), 'года')

Ожидаемая продолжительность жизни: 74 года


Предсказание на 2020

In [40]:
x_ = x_ + 1
y_ = np.array(y[1:])

In [41]:
ensemble_rf = []
for i in range (100):
    ensemble_rf.append(RandomForestRegressor(n_estimators=50, random_state=i).fit(x_, y_))

In [42]:
ensemble_et = []
for i in range (100):
    ensemble_et.append(ExtraTreesRegressor(n_estimators=50, random_state=i).fit(x_, y_))

In [44]:
ensemble_cb = []
for i in range (100):
    ensemble_cb.append(CatBoostRegressor(iterations=20, depth=16, random_state=i, verbose=False).fit(x_, y_))

In [45]:
prediction = 0
for i in range (100):
    prediction += ensemble_rf[i].predict(x[-1:])
    prediction += ensemble_et[i].predict(x[-1:])
    prediction += ensemble_cb[i].predict(x[-1:])
prediction /= 300
print (prediction)

[0.24241265]


In [46]:
data_predict = data.copy()
data_predict[y_column] = np.ones(len(data_predict)) * prediction
data_predict = scaler.inverse_transform(data_predict)
answer20 = data_predict[0][np.where(features==y_column)][0]
print('Ожидаемая продолжительность жизни:', round(answer20), 'года')

Ожидаемая продолжительность жизни: 69 года
